# チューニング【画像のCROP】
画像のパターンを増やすため、画像をCROPしパターンを増加させる
## スマホにて画像撮影を実施し、該当イメージをアップロードする
* アップロードディレクトリを作成する

In [ ]:
!apt-get update && apt-get upgrade -y
!apt-get install libgl1-mesa-dev -y
!pip install boto3
!pip install opencv-python pyheif

import sagemaker, json, numpy as np, os, boto3, uuid
from PIL import Image, ImageDraw, ImageOps, ImageColor, ImageFont
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from glob import glob
import cv2
import pyheif
np.random.seed(seed=1234)
print('---------- 終 了 ----------')

In [ ]:
# S3 からマニフェストファイルをLocal（manifestフォルダ）へダウンロード
LOCAL_MANIFEST_DIR = "./crop_manifest/"
# BASE_PREFIX = "kinotake/images"
GT_JOB_NAME = "team-z-job3"
# 各自のS3バケットを指定する",
S3_BUCKET='training-menter1-s3'
# 【変更不要】imageファイルの格納場所指定,
IMAGES_PATH='training2/images'
sagemaker.session.Session().download_data(LOCAL_MANIFEST_DIR, key_prefix=f'{IMAGES_PATH}/{GT_JOB_NAME}/manifests/output/output.manifest',bucket=S3_BUCKET)

In [ ]:
# S3 からLocalへラベリング済みデータのダウンロード
TRAIN_RAWIMAGE_DIR = './crop_train_raw_images/'
import shutil
try:
    shutil.rmtree(TRAIN_RAWIMAGE_DIR)
except OSError as err:
    pass
os.mkdir(TRAIN_RAWIMAGE_DIR)

# ラベリング結果をテキストとして読み込む
with open(f'{LOCAL_MANIFEST_DIR}/output.manifest','r') as f:
    manifest_line_list = f.readlines()

for manifest_line in manifest_line_list:
    # 画像のラベリング結果の読み込み
    manifest_dict = json.loads(manifest_line)
    filename = manifest_dict['source-ref'].split('/')[-1]
    print(filename)
    sagemaker.session.Session().download_data(TRAIN_RAWIMAGE_DIR, key_prefix=f'{IMAGES_PATH}/{filename}',bucket=S3_BUCKET)


In [ ]:
# クロップした画像にきのこの山やたけのこの里が映っている場合、
# クロップした後のきのこの山やたけのこの里が1/4以下かどうかを判定するヘルパー関数

def fix_bbox(l,t,r,b,w,h):
    # 判定結果、NG なら False にする
    judge = True
    # ラベリング結果のクロップ補正後の値が負の値ならば 0 に、イメージサイズより大きければイメージサイズに補正する
    fix_left = 0 if l < 0 else l
    fix_top = 0 if t < 0 else t
    fix_right = w if r > w else r
    fix_bottom = h if b > h else b
    # 領域外ならラベリング無しとする
    if l > w or t > h or r < 0 or b <0:
        judge=False
    # 基の面積の1/4以下ならアノテーション無しとする
    elif (r-l)*(b-t)/4 > (fix_right-fix_left)*(fix_bottom-fix_top):
        judge=False
    
    return judge,(fix_left,fix_top,fix_right,fix_bottom)
    

In [ ]:
OUTPUT_DIR = './crop_train_random_crop_images/'
try:
    shutil.rmtree(OUTPUT_DIR)
except OSError as err:
    pass
os.mkdir(OUTPUT_DIR)

# ラベリング結果をテキストとして読み込む
with open('crop_manifest/output.manifest','r') as f:
    manifest_line_list = f.readlines()

# クロップサイズの定数
IMAGE_SIZE_TUPLE=(200,200)

# 結果のきのこの山やたけのこの里の位置情報を格納する辞書 
annotation_dict = {
    'images':[],
    'annotations':[]
}

# 画像のファイル名に使う一意なシーケンス番号
IMAGE_ID = 0

# ラベリング結果の行数分ループする
# ラベリング結果は 1 行につき 1 画像格納される
for manifest_line in manifest_line_list:
    # 画像のラベリング結果の読み込み
    manifest_dict = json.loads(manifest_line)
    # 画像のファイル名取得(ラベリング結果に格納されている)
    filename = manifest_dict['source-ref'].split('/')[-1]
    annotation_list = manifest_dict[GT_JOB_NAME]['annotations']
    # 元画像のサイズを取得(ラベリング結果に格納されている)
    image_size_tuple=(manifest_dict[GT_JOB_NAME]['image_size'][0]['width'],manifest_dict[GT_JOB_NAME]['image_size'][0]['height'])
    # PIL で画像を開く
    raw_img = Image.open(os.path.join(TRAIN_RAWIMAGE_DIR,filename))
    # 20 回クロップする
    for i in range(20):
        # ループするかどうかのフラグ(画像にきのこの山やたけのこの里が 2 枚未満だったらクロップをやりなおし)
        loop = True
        while loop:
            # クロップを行う左上の座標を設定
            rand_x = np.random.randint(0,image_size_tuple[0]-IMAGE_SIZE_TUPLE[0])
            rand_y = np.random.randint(0,image_size_tuple[1]-IMAGE_SIZE_TUPLE[1])
            # クロップする
            crop_img = raw_img.crop((
                rand_x,
                rand_y,
                rand_x + IMAGE_SIZE_TUPLE[0],
                rand_y + IMAGE_SIZE_TUPLE[1]
            ))
            # クロップ後のきのこの山やたけのこの里の位置を格納するリスト
            annotation_list = []
            # 元画像のラベリング結果をループ
            for annotation in manifest_dict[GT_JOB_NAME]['annotations']:
                # クロップした後のきのこの山やたけのこの里の座標に補正
                left = annotation['left'] - rand_x
                top = annotation['top'] - rand_y
                right = annotation['left'] + annotation['width'] - rand_x
                bottom = annotation['top'] + annotation['height'] - rand_y
                # きのこの山やたけのこの里があるかどうかを判定
                judge,(left,top,right,bottom) = fix_bbox(left,top,right,bottom,IMAGE_SIZE_TUPLE[0],IMAGE_SIZE_TUPLE[1])
                if judge:
                    # きのこの山やたけのこの里があったら位置とラベルを追加
                    annotation_list.append(
                        {
                            'bbox':[left,top,right,bottom],
                            'category_id':annotation['class_id']
                        }
                    )
            # きのこの山やたけのこの里と数が2未満だったらクロップやり直し
            if len(annotation_list) > 0:
                loop = False
        
        # クロップしたら画像を保存する
        save_file_name = f'{str(IMAGE_ID).zfill(5)}_{str(i).zfill(5)}_{filename}'.replace('jpg','png')
        crop_img.save(os.path.join(OUTPUT_DIR,save_file_name))
        
        # 補正済ラベリング結果を出力用辞書に格納
        annotation_dict['images'].append(
            {
                'file_name' : save_file_name,
                'height' : IMAGE_SIZE_TUPLE[1],
                'width' : IMAGE_SIZE_TUPLE[0],
                'id' : IMAGE_ID
            }
        )
        for annotation in annotation_list:                  
            annotation_dict['annotations'].append(
                {
                    'image_id': IMAGE_ID,
                    'bbox':annotation['bbox'],
                    'category_id':annotation['category_id']
                }
            )
        IMAGE_ID += 1
    # # print(annotation_list)
    # # ラベリング結果を出力用辞書に格納
    # annotation_dict['images'].append(
    #     {
    #         'file_name' : filename,
    #         'height' : manifest_dict['kinotake-user1']['image_size'][0]['height'],
    #         'width' : manifest_dict['kinotake-user1']['image_size'][0]['width'],
    #         'id' : IMAGE_ID
    #     }
    # )
    # for annotation in annotation_list:
    #     # 座標変換
    #     left = annotation['left']
    #     top = annotation['top']
    #     right = annotation['left'] + annotation['width']
    #     bottom = annotation['top'] + annotation['height']
    #     # アノテーションを編集
    #     annotation_dict['annotations'].append(
    #         {
    #             'image_id': IMAGE_ID,
    #             'bbox': [left, top, right, bottom],
    #             # 文字列へ変換し格納
    #             # 'category_id': manifest_dict['kinotake-user1-metadata']['class-map'][str(annotation['class_id'])]
    #             'category_id': annotation['class_id']
    #         }
    #     )
    
    # IMAGE_ID += 1
    
# ランダムクロップ補正後のラベリング結果を出力
with open('annotations.json','wt') as f:
    f.write(json.dumps(annotation_dict))        


# 出力したディレクトリを prefix として使う
prefix = OUTPUT_DIR[2:-1]

# # re-run 用の削除コマンド
# !aws s3 rm s3://{S3_BUCKET}/{prefix} --recursive
# # ランダムクロップした画像をアップロード    
# image_s3_uri = sagemaker.session.Session().upload_data(OUTPUT_DIR,key_prefix=f'{prefix}/images')
# # ラベリング結果をアップロード
# annotatione_s3_uri = sagemaker.session.Session().upload_data('./annotations.json',key_prefix=prefix)
# # Fine-Tune で使う URI を出力
# paste_str = image_s3_uri.replace('/images','')
# print(f"paste string to S3 bucket address：{paste_str}")
CROP_DIR=f'./{prefix}'
# print(CROP_DIR)
!aws s3 rm s3://{S3_BUCKET}/training2/{prefix} --recursive
!aws s3 cp {CROP_DIR} s3://{S3_BUCKET}/training2/{prefix}/images --exclude \".*\"  --recursive
!aws s3 cp ./annotations.json s3://{S3_BUCKET}/training2/{prefix}/
print('---------- 終 了 ----------')

# CROP結果の確認
* FILE_NAMEへ対象ファイル名を指定し実行すると（CROP画像が確認できます）

In [ ]:
# 入力画像確認
FILE_NAME = '00334_00014_IMG_0218.png'
INPUT_IMAGE_FILE = f'{OUTPUT_DIR}{FILE_NAME}'
print(INPUT_IMAGE_FILE)
# 推論対象の画像を開いて変数に格納
with open(INPUT_IMAGE_FILE, 'rb') as f:
    img_bin = f.read()

# ラベリング結果(json)を読み込む
with open('annotations.json') as f:
    annotations_json = json.load(f)

image_id = 0
plt.clf()
plt.close()

# 対象イメージを特定
for annotations_image in annotations_json['images']:
    if annotations_image['file_name'] == FILE_NAME:
        print(annotations_image)
        image_id = annotations_image['id']

image_np = np.array(Image.open(INPUT_IMAGE_FILE))
# matplotlibで描画する
fig = plt.figure(figsize=(20,20))
ax = plt.axes()
ax.imshow(image_np)
# イメージに対するアノテーションを取得
for annotations_row in annotations_json['annotations']:
    if annotations_row['image_id'] == image_id:
        # 検出した座標（左上を(0,0),右下を(1,1)とした相対座標)を取得
        left, top, right, bot = annotations_row['bbox']
         # 相対座標を絶対座標に変換する
        x = left
        w = right - left
        y = bot
        h = top - bot
         # 検出した物体の ID を take/kino に読み替える
        class_name = 'kino' if int(annotations_row['category_id'])==0 else 'take'
         # take/kinoに対して矩形で描画するための色を設定する
        color = 'blue' if class_name == 'take' else 'red'
         # matplotlib に検出した物体に矩形を描画する
        rect = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        # 左上に検出結果と信頼度スコアを描画する
        # ax.text(x, y, "{} {:.0f}%".format(class_name, 0), bbox=dict(facecolor='white', alpha=0.5))
        ax.text(x, y, "{}".format(class_name), bbox=dict(facecolor='white', alpha=0.5))

fig